## Udacity Set to Coco

In [1]:
import sys
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')
print(sys.path)

['', '/home/dean/anaconda3/envs/ros-kache/lib/python35.zip', '/home/dean/anaconda3/envs/ros-kache/lib/python3.5', '/home/dean/anaconda3/envs/ros-kache/lib/python3.5/plat-linux', '/home/dean/anaconda3/envs/ros-kache/lib/python3.5/lib-dynload', '/home/dean/.local/lib/python3.5/site-packages', '/home/dean/anaconda3/envs/ros-kache/lib/python3.5/site-packages', '/home/dean/anaconda3/envs/ros-kache/lib/python3.5/site-packages/IPython/extensions', '/home/dean/.ipython']


In [2]:
## Imports

%matplotlib inline
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
from PIL import Image

import scipy.misc
from skimage import measure   

import time
import os
import zipfile as zf
import tarfile
import csv
import pickle
import urllib
import pandas as pd

/home/dean/anaconda3/envs/ros-kache/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [3]:
## Dataset Parameters ##
TRAINING_DATASET_DIRECTORY = 'training-set/'
WORKING_DIRECTORY = '/media/dean/49D9D6EB1BE53FC4/datasets/UdacityDrive/object-detection-crowdai/'
OBJ_NEGATIVES_TOKEN = 'non_objects'
dataset_path = "{}{}{}{}".format(WORKING_DIRECTORY, TRAINING_DATASET_DIRECTORY,'**/', '*.png')
DATACACHE_DIRECTORY = os.path.join(WORKING_DIRECTORY, 'datacache/')


## Udacity Dataset Extraction Parameters ##
UDACITY_ANNS_CSV = os.path.join(WORKING_DIRECTORY, 'labels.csv')
HEADER_ROW=['xstart', 'ystart', 'xstop', 'ystop', 'frame', 'label', 'preview_url']
UDACITY_SOURCE_URL = 'http://bit.ly/udacity-annoations-crowdai'
DATASET_ZIPFILE = 'object-detection-crowdai.tar.gz'
OVERWRITE_UDACITY_DATASET = False
UDACITY_DATASET_DIRECTORY = os.path.join(WORKING_DIRECTORY, TRAINING_DATASET_DIRECTORY, 'udacity-set') 

## RoadCOCO Dataset Labels
RoadCOCO_LABELS_PATH = os.path.join(WORKING_DIRECTORY, '../','RoadCOCO_Classes.csv')
GT_HEADER_ROW = ['class', 'super-category', 'special', 'description']

## Image Processing ##
DEFAULT_WIDTH, DEFAULT_LENGTH, DEFAULT_DEPTH = (448, 448, 3)
DEFAULT_RESOLUTION = (DEFAULT_LENGTH, DEFAULT_WIDTH, DEFAULT_DEPTH)
if DEFAULT_DEPTH == 1:
    DEFAULT_RESOLUTION = DEFAULT_RESOLUTION[:-1]
    
## Training Parameters ##
OVERWRITE_DATACACHE = True

### Helper Functions - Udacity Dataset Loading & Augmentation

In [4]:
# Define a function to scale .PNG and JPEG Files both to 0 to 1 
def normalize_pixels(img):
    max_pixel_value = np.max(img)
    if max_pixel_value > 1.0:
        img = np.copy(np.subtract(np.divide(img, 255.0), 0.5)).astype(np.float64)
    return img

# Define a function to scale .PNG and JPEG Files both to 0 to 1 
def denormalize_pixels(img):    
    img = np.copy(np.multiply(np.add(img,0.5), 255.0)).astype(np.int32)  
    assert np.min(img) >= 0
    assert np.max(img) <= 255
    return img

def process_img(filepath):
    image = cv2.imread(filepath)
    img_copy = image.copy()
    img_copy = cv2.cvtColor(img_copy,cv2.COLOR_BGR2RGB)
    img_copy = img_copy/255.-.5 
    return img_copy

In [5]:
def maybe_download(source_url, filename):
    if not os.path.exists(WORKING_DIRECTORY):
        os.mkdir(WORKING_DIRECTORY)
    
    filepath = os.path.join(WORKING_DIRECTORY, filename)
    if not os.path.exists(filepath):
        filepath, _ = urllib.request.urlretrieve(source_url, filepath)
        statinfo = os.stat(filepath)
        print('')
        print('Succesfully downloaded:', filepath, '| % d MB.' % int(statinfo.st_size*1e-6))
    return filepath

In [6]:
def unzip_file(zip_file, source_dir_name=None, destination=WORKING_DIRECTORY):
    if 'tar.gz' in zip_file:
        head, tail = os.path.splitext(zip_file)
        if not os.path.exists(os.path.join(os.path.splitext(head)[0])):
            print('unzipping file:', zip_file, 'to directory:', os.path.join( os.path.splitext(head)[0]))
            tar = tarfile.open(zip_file, "r:*")
            tar.extractall(destination)
            tar.close()
    else: #.zip extension
        head, tail = os.path.splitext(zip_file)
        if not os.path.exists(os.path.join(destination, head)):
            print('File does not exist: ', os.path.join(destination, head), ': Extracting')
            zipf = zf.ZipFile(os.path.join(WORKING_DIRECTORY,zip_file))
            print('Loaded zipf',zipf, ': Extracting')
            zipf.extractall(os.path.join(destination, head))
            zipf.close()

In [7]:
def extract_and_preprocess_image(filepath, objs_dir, nonobjs_dir, xstart, ystart, xstop, ystop, obj_class,
                                 img_size=(DEFAULT_LENGTH, DEFAULT_WIDTH), img_ext = '.png'):
    full_path = os.path.join(WORKING_DIRECTORY, 'object-detection-crowdai', filepath)    
    # Image read in from cv2 + .jpg -> (0 to 1)
    if os.path.exists(full_path) or OVERWRITE_UDACITY_DATASET == True:
        # Use cv2 to open image and extract bounding boxes
        img = process_img(full_path)
        img_height, img_width = img.shape[:2]
        # Extract Image. Note: numpy arrays are (row, col)!
        
        # Add Offests to allow for Object Detection Tasks
        offset = 20
        xstart_mod, ystart_mod, xstop_mod, ystop_mod =  (0 if xstart-offset <= 0 else xstart-offset), \
                                                        (0 if ystart-offset <= 0 else ystart-offset), \
                                                        (img_width if xstop+offset > img_width else xstop+offset), \
                                                        (img_height if ystop+offset > img_height else ystop+offset)
            
        detection = img[ystart_mod:ystop_mod, xstart_mod:xstop_mod]
        resized_detection = cv2.resize(detection, img_size, interpolation=cv2.INTER_AREA)
        denorm = denormalize_pixels(resized_detection)        

                
        # Save Detected Object Image with offsets
        filename, ext = os.path.splitext(filepath)
        new_filename =  "{}_{}_{}_{}_{}_{}{}".format(filename,obj_class, xstart, ystart, xstop, ystop, img_ext)
        scipy.misc.imsave(os.path.join(objs_dir, new_filename), denorm)
        

        ## (Optional) Auto-Generate 'Negative Class' Labels to keep dataset balanced
        i_lrc = np.random.randint(3) # 66% chance of auto generating non-obj features
        if (i_lrc > 1):
            try:
                # Estimate Negative Label as crop surrounding the Positive Detections
                xstart_mod, ystart_mod, xstop_mod, ystop_mod = xstart, \
                                                        (0 if ystart-(ystop-ystart) < 0 else ystart-(ystop-ystart)), xstop, ystart
                new_filename =  "{}_{}_{}_{}_{}_{}{}".format(filename,'non_'+obj_class.lower(), xstart_mod, ystart_mod, xstop_mod, ystop_mod, img_ext)
                path_file = os.path.join(nonobjs_dir, new_filename)


                nonobj_img = img[ystart_mod:ystop_mod, xstart_mod:xstop_mod]
                resized_nonobj_img = cv2.resize(nonobj_img, img_size, interpolation=cv2.INTER_AREA)

                # Save Negative Label
                scipy.misc.imsave(path_file, resized_nonobj_img)
            except:
                print("Error saving negative label:", path_file, " Moving on..")

In [8]:
## Download Udacity Dataset
tar_file = maybe_download( UDACITY_SOURCE_URL, os.path.join(WORKING_DIRECTORY,DATASET_ZIPFILE))
unzip_file(tar_file)
                    
cars_dir = os.path.join(UDACITY_DATASET_DIRECTORY, 'cars')
trucks_dir = os.path.join(UDACITY_DATASET_DIRECTORY, 'trucks')
peds_dir = os.path.join(UDACITY_DATASET_DIRECTORY, 'pedestrians')

# Make Directory for Negative Annotations
nonobjs_dir = os.path.join(UDACITY_DATASET_DIRECTORY, OBJ_NEGATIVES_TOKEN)
noncars_dir = os.path.join(nonobjs_dir, 'non_cars')
nontrucks_dir = os.path.join(nonobjs_dir, 'non_trucks')
nonpeds_dir = os.path.join(nonobjs_dir, 'non_peds')

# Create Directories to store images for Pre-processing
os.makedirs(cars_dir, exist_ok=True)
os.makedirs(trucks_dir, exist_ok=True)
os.makedirs(peds_dir, exist_ok=True)
os.makedirs(noncars_dir, exist_ok=True)
os.makedirs(nontrucks_dir, exist_ok=True)
os.makedirs(nonpeds_dir, exist_ok=True)

In [9]:
## Extract Labels ##
annotations = pd.read_csv(UDACITY_ANNS_CSV, names=HEADER_ROW, skiprows=1)
annotations.head()

,xstart,ystart,xstop,ystop,frame,label,preview_url
0,785,533,905,644,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
1,89,551,291,680,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
2,268,546,383,650,1479498371963069978.jpg,Car,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
3,455,522,548,615,1479498371963069978.jpg,Truck,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...
4,548,522,625,605,1479498371963069978.jpg,Truck,http://crowdai.com/images/Wwj-gorOCisE7uxA/vis...


In [10]:
# ## Extract Labels ## - Skip for Now      
# for label in annotations.as_matrix():
#     filename, ext = os.path.splitext(label[4])
#     new_filename =  "{}_{}_{}_{}_{}_{}{}".format(filename,label[5].lower(), label[0], label[1], label[2], label[3], '.png')
    
#     for obj_class in set(annotations['label'].tolist()):
#         if not os.path.exists(os.path.join(cars_dir, new_filename)) and label[5].lower() == obj_class.lower():
#             extract_and_preprocess_image(label[4], cars_dir, noncars_dir, xstart=label[0], ystart=label[1],
#                                          xstop=label[2], ystop=label[3], obj_class=obj_class)
#         elif not os.path.exists(os.path.join(trucks_dir, new_filename)) and label[5].lower() == obj_class.lower():
#             extract_and_preprocess_image(label[4], trucks_dir, nontrucks_dir, xstart=label[0], ystart=label[1],
#                                          xstop=label[2], ystop=label[3], obj_class=obj_class)
#         elif not os.path.exists(os.path.join(peds_dir, new_filename)) and label[5].lower() == obj_class.lower():
#             extract_and_preprocess_image(label[4], peds_dir, nonpeds_dir, xstart=label[0], ystart=label[1],
#                                          xstop=label[2], ystop=label[3], obj_class=obj_class)

In [11]:
# Get RoadCOCO Labels to Use as Ground Truth
gt_labels = pd.read_csv(RoadCOCO_LABELS_PATH, names=GT_HEADER_ROW, skiprows=1)
gt_labels.head()

,class,super-category,special,description
0,Vehicle,Vehicle,Orientation matters,Combination of all other vehicles to get a cat...
1,Car,Vehicle,Orientation matters,NaN
2,Pickup,Truck,Orientation matters,NaN
3,TractorSemi,Truck,Orientation matters,NaN
4,TrailerSemi,Truck,Orientation matters,NaN


In [12]:
# Represent Category IDs using RoadCOCO Labels
cats2ids = {}
for i, label in enumerate(sorted(set(gt_labels['class'].tolist()))):
    cats2ids[str(label).lower()] = i
ids2cats = {i: v for v, i in cats2ids.items()}
    


# Build Categories List in MS RoadCOCO Format
coco_categories = [] 
for label in gt_labels.as_matrix():
    category = str(label[0]).lower()
    cat_id = cats2ids[category]
    
    
    # Verify super-category in list, handle edge cases
    if str(label[1]).lower() == 'truck':
        sup_cat = ids2cats[cats2ids['utilitytruck']]
    elif str(label[1]).lower() == 'constructionfurniture':
        sup_cat = ids2cats[cats2ids['constructionequipment']]
    elif str(label[1]).lower() == 'pedestrian':
        sup_cat = ids2cats[cats2ids['person']]
    else:
        sup_cat = ids2cats[cats2ids[str(label[1]).lower()]]
    
    coco_categories.append({"id": cat_id, "name": category, "supercategory":sup_cat})   
print (coco_categories)

[{'id': 62, 'name': 'vehicle', 'supercategory': 'vehicle'}, {'id': 8, 'name': 'car', 'supercategory': 'vehicle'}, {'id': 37, 'name': 'pickup', 'supercategory': 'utilitytruck'}, {'id': 52, 'name': 'tractorsemi', 'supercategory': 'utilitytruck'}, {'id': 60, 'name': 'trailersemi', 'supercategory': 'utilitytruck'}, {'id': 21, 'name': 'firetruck', 'supercategory': 'utilitytruck'}, {'id': 0, 'name': 'ambulance', 'supercategory': 'utilitytruck'}, {'id': 39, 'name': 'police', 'supercategory': 'vehicle'}, {'id': 10, 'name': 'constructionequipment', 'supercategory': 'constructionequipment'}, {'id': 34, 'name': 'motorcycle', 'supercategory': 'vehicle'}, {'id': 43, 'name': 'scooter', 'supercategory': 'person'}, {'id': 4, 'name': 'bicyclist', 'supercategory': 'vehicle'}, {'id': 20, 'name': 'escooter', 'supercategory': 'vehicle'}, {'id': 45, 'name': 'skateboard', 'supercategory': 'misc_label'}, {'id': 64, 'name': 'wheelchair', 'supercategory': 'person'}, {'id': 51, 'name': 'stroller', 'supercategory

/home/dean/anaconda3/envs/ros-kache/lib/python3.5/site-packages/ipykernel/__main__.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
# Script to convert Udacity Drive Annotations to Coco Json Annotations
import xml.etree.cElementTree as ET
import glob
import argparse
import os
import numpy as np
import json
import numpy
import cv2
from collections import OrderedDict
from shapely.geometry import Polygon, MultiPolygon

from PIL import Image
annotations = pd.read_csv(UDACITY_ANNS_CSV, names=HEADER_ROW, skiprows=1)


images, anns = [], []
img_paths = set(annotations['frame'].tolist())

num_imgs = len(img_paths)
ann_index = 0
    
for img_id, f in enumerate(sorted(img_paths)):
    im = Image.open(os.path.join(WORKING_DIRECTORY, f))
    width, height = im.size
    dic = {'file_name': f, 'id': img_id, 'height': height, 'width': width}
    images.append(dic)
    binary_mask = np.zeros_like(im)[:,:,0]
    
    for annotation in [x for x in annotations.as_matrix() if x[4].lower() == f.lower()]:
        # xy coords: [xstart, ystart, xstop, ystop] -> bbox = [x,y,width,height]
        xstart, ystart, xstop, ystop = annotation[0], annotation[1], \
                                       annotation[2], annotation[3]        
        binary_mask[xstart:xstop, ystart:ystop] +=1
        
        
    contours = measure.find_contours(binary_mask, 0.5, positive_orientation='low')
    polygons = []
    segmentations = []

    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        # for i in range(len(contour)):
        #     row, col = contour[i]
        #     contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)

        if poly and poly.exterior:
            polygons.append(poly)
            segmentation = np.array(poly.exterior.coords).ravel().tolist()
            segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    if multi_poly.bounds:
        x, y, max_x, max_y = multi_poly.bounds
        width = max_x - x
        height = max_y - y
        bbox = (x, y, width, height)
        area = multi_poly.area

        # Get Label
        if annotation[5].lower() in cats2ids.keys():
            category = cats2ids[annotation[5].lower()]
        else:
            # Flag data point for relabeling
            category = cats2ids['misc_label']

        annotation = {
            'segmentation': segmentations,
            'iscrowd': 0,
            'image_id': img_id,
            'category_id': category,
            'id': ann_index,
            'bbox': bbox,
            'area': area
        }
        ann_index+=1
        anns.append(annotation) 

/home/dean/anaconda3/envs/ros-kache/lib/python3.5/site-packages/ipykernel/__main__.py:30: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [ ]:
print(anns[:10])

In [ ]:
from datetime import datetime
INFO = {
    "description": "Road Object-Detections Dataset based on MS COCO",
    "url": "https://kache.ai",
    "version": "0.0.1",
    "year": 2018,
    "contributor": "deanwebb",
    "date_created": datetime.utcnow().isoformat(' ')
}

LICENSES = [
    {
        "id": 1,
        "name": "The MIT License (MIT)",
        "url": "https://opensource.org/licenses/MIT",
        "description":  """
                        The MIT License (MIT)
                        Copyright (c) 2017 Matterport, Inc.

                        Permission is hereby granted, free of charge, to any person obtaining a copy
                        of this software and associated documentation files (the "Software"), to deal
                        in the Software without restriction, including without limitation the rights
                        to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
                        copies of the Software, and to permit persons to whom the Software is
                        furnished to do so, subject to the following conditions:

                        The above copyright notice and this permission notice shall be included in
                        all copies or substantial portions of the Software.

                        THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
                        IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
                        FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
                        AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
                        LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
                        OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
                        THE SOFTWARE.
                        """
    }
]

coco_output = {'info': INFO, 'licenses': LICENSES, 'images':images, 'annotations':anns, 'categories': coco_categories}
with open('{}/instances_shape_train2018.json'.format(WORKING_DIRECTORY), 'w') as output_json_file:
    json.dump(coco_output, output_json_file)

In [ ]:
%matplotlib inline
from pycocotools.coco import COCO
import numpy as np
import skimage.io as io
import matplotlib.pyplot as plt
import pylab

In [ ]:
image_directory = WORKING_DIRECTORY
annotation_file = os.path.join(WORKING_DIRECTORY,'instances_shape_train2018.json')

In [ ]:
example_coco = COCO(annotation_file)

In [ ]:
categories = example_coco.loadCats(example_coco.getCatIds())
category_names = [category['name'] for category in categories]
print('Custom COCO categories: \n{}\n'.format(' '.join(category_names)))

category_names = set([category['supercategory'] for category in categories])
print('Custom COCO supercategories: \n{}'.format(' '.join(category_names)))

In [ ]:
category_ids = example_coco.getCatIds(catNms=['square'])
image_ids = example_coco.getImgIds(catIds=category_ids)
image_data = example_coco.loadImgs(image_ids[np.random.randint(0, len(image_ids))])[0]
print(image_data)

In [ ]:
# load and display instance annotations
image = io.imread(os.path.join(image_directory ,image_data['file_name']))
plt.imshow(image); plt.axis('off')
pylab.rcParams['figure.figsize'] = (68.0, 120.0)
annotation_ids = example_coco.getAnnIds(imgIds=image_data['id'], catIds=category_ids, iscrowd=None)
annotations = example_coco.loadAnns(annotation_ids)
example_coco.showAnns(annotations)